In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

%load_ext autoreload

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

data = xr.open_dataset("../1_data_processing/1_1_Forcing/products/all_stations_cmems.zarr", engine="zarr")
data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
data.time.attrs["axis"] = "T"
data = data.sel(time=slice(time_start, time_end))
_ = data.load()

cafe_npp = xr.open_dataset("../1_data_processing/1_1_Forcing/products/all_stations_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time", how="all")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.time.attrs["axis"] = "T"
_ = cafe_npp.load()

In [3]:
data

<xarray.Dataset> Size: 13MB
Dimensions:              (latitude: 3, longitude: 3, time: 8767, depth: 3)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 24B 22.75 31.6 50.0
  * longitude            (longitude) float64 24B -158.0 -150.0 -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 2MB 23.52...
    U                    (latitude, longitude, time, depth) float64 2MB 0.016...
    V                    (latitude, longitude, time, depth) float64 2MB -0.07...
    mnkc_epi             (latitude, longitude, time) float64 631kB 0.269 ... nan
    mnkc_hmlmeso         (latitude, longitude, time) float64 631kB 0.7915 ......
    mnkc_lmeso           (latitude, longitude, time) float64 631kB 2.173 ... nan
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 631kB 0.4889 ......
    mnkc_umeso           (latitude, longitude, time) float64 631kB 0.4931 ......
    npp                  (latitude, longitude, time) float64 631kB 315.7 ... nan
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 2MB 117.3...
    zeu                  (latitude, longitude, time) float64 631kB 78.19 ... nan
    zooc                 (latitude, longitude, time) float64 631kB 0.2896 ......

In [4]:
cafe_npp

<xarray.Dataset> Size: 701kB
Dimensions:    (time: 8767, latitude: 3, longitude: 3)
Coordinates:
  * latitude   (latitude) float64 24B 22.75 31.6 50.0
  * longitude  (longitude) float64 24B -158.0 -150.0 -64.2
  * time       (time) datetime64[ns] 70kB 1998-01-01 1998-01-02 ... 2022-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 631kB 306.4 nan ... nan nan
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [5]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../1_data_processing/1_1_Forcing/products/Bats_obs.zarr", engine="zarr")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)

obs_hot = xr.open_dataset("../1_data_processing/1_1_Forcing/products/Hot_obs.zarr", engine="zarr")
obs_hot = update_layer(obs_hot, epipelagic_size)
obs_hot = Observation(obs_hot)

obs_papa = xr.open_dataset("../1_data_processing/1_1_Forcing/products/Papa_obs.zarr", engine="zarr")
obs_papa = update_layer(obs_papa, epipelagic_size)
obs_papa = Observation(obs_papa)

observations = [obs_bats, obs_hot, obs_papa]

Create structure for SeapoPym simulation.


In [6]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=data["T"], resolution=1 / 12, timestep=1),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12, timestep=1),
)

2024-11-21 14:16:55,150 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [7]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        # tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        # tr_max=Parameter("D1N1_tr_max", 0, 50),
        tr_rate=-0.11,
        tr_max=10.38,
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        # tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        # tr_max=Parameter("D2N1_tr_max", 0, 50),
        tr_rate=-0.11,
        tr_max=10.38,
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [8]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [9]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    ETA=0.5,
    INDPB=0.2,
    CXPB=0.2,
    MUTPB=0.2,
    NGEN=15,
    POP_SIZE=300,
    cost_function_weight=(-(1 / 3), -(1 / 3), -(1 / 3)),
    hall_of_fame_size=1000,
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [10]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [11]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64069 instead
  warnings.warn(


And watch the magic on the Dask dashboard :


In [12]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:64069/status,
Dashboard: http://127.0.0.1:64069/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64070,Workers: 4
Dashboard: http://127.0.0.1:64069/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:64083,Total threads: 2
Dashboard: http://127.0.0.1:64084/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:64073,


And execute the process.


In [13]:
viewer = genetic_algo.optimize()

2024-11-21 14:16:56,913 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 14:16:56,913 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 14:16:56,913 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 14:16:56,914 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 14:16:56,915 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-11-21 14:16:56,915 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 14:16:56,916 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 14:16:56,917 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 14:16:56,918 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 14:16:56,925 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-11-21 14:16:56,925 :: Seapodym ::  DEBUG ::
|	Direct co

Finaly here is the result :


In [19]:
viewer.logbook

D1N1_inv_lambda_max  D1N1_inv_lambda_rate  \
generation individual                                              
0          0                     34.820291              0.099060   
           1                     13.132037              0.280213   
           2                     90.573450              0.292873   
           3                     86.644612              0.941901   
           4                     63.787658              0.074120   
...                                    ...                   ...   
15         295                   99.897433              0.130669   
           296                   98.133104              0.126855   
           297                   99.981984              0.130669   
           298                   99.897433              0.130669   
           299                   99.089259              0.130669   

                       D1N1_energy_coefficient  D2N1_inv_lambda_max  \
generation individual                                                 
0          0                          0.117494             1.619370   
           1                          0.228488            51.269857   
           2                          0.727742            26.938805   
           3                          0.070409            54.591208   
           4                          0.278146            99.277802   
...                                        ...                  ...   
15         295                        0.162374            40.797430   
           296                        0.162374            40.797430   
           297                        0.162374            40.797430   
           298                        0.162374            40.797430   
           299                        0.162374            40.797430   

                       D2N1_inv_lambda_rate  D2N1_energy_coefficient   fitness  
generation individual                                                           
0          0                       0.263246                 0.593262  0.242701  
           1                       0.172986                 0.227828  0.379177  
           2                       0.084374                 0.072424  0.231410  
           3                       0.207342                 0.315440  0.456789  
           4                       0.926813                 0.405706  1.013091  
...                                     ...                      ...       ...  
15         295                     0.177558                 0.571831  0.124757  
           296                     0.177558                 0.571831  0.124122  
           297                     0.177558                 0.571831  0.124742  
           298                     0.177558                 0.571831  0.124757  
           299                     0.177558                 0.571831  0.124909  

[4800 rows x 7 columns]

In [20]:
viewer.hall_of_fame

D1N1_inv_lambda_max  D1N1_inv_lambda_rate  \
generation individual                                              
9          33                    98.133104              0.130669   
15         103                   98.133104              0.126855   
           229                   98.133104              0.126855   
           267                   98.133104              0.126855   
12         140                   98.133104              0.126855   
...                                    ...                   ...   
0          77                    53.570806              0.008383   
4          166                   85.751749              0.043586   
10         159                   98.133104              0.013744   
5          130                   61.192722              0.006699   
0          239                   67.264858              0.930777   

                       D1N1_energy_coefficient  D2N1_inv_lambda_max  \
generation individual                                                 
9          33                         0.162374            79.946924   
15         103                        0.162374            40.797430   
           229                        0.162374            40.797430   
           267                        0.162374            40.797430   
12         140                        0.162374            40.797430   
...                                        ...                  ...   
0          77                         0.491223            82.767245   
4          166                        0.654705            40.797430   
10         159                        0.359609            35.518938   
5          130                        0.560931            35.229170   
0          239                        0.191813            62.924785   

                       D2N1_inv_lambda_rate  D2N1_energy_coefficient  \
generation individual                                                  
9          33                      0.084374                 0.072424   
15         103                     0.177558                 0.571831   
           229                     0.177558                 0.571831   
           267                     0.177558                 0.571831   
12         140                     0.177558                 0.571831   
...                                     ...                      ...   
0          77                      0.431845                 0.369207   
4          166                     0.177558                 0.108410   
10         159                     0.530087                 0.571831   
5          130                     0.530087                 0.397982   
0          239                     0.006388                 0.777621   

                          fitness  
generation individual              
9          33            0.122639  
15         103           0.124122  
           229           0.124122  
           267           0.124122  
12         140           0.124122  
...                           ...  
0          77           63.471250  
4          166          67.109544  
10         159          90.186617  
5          130         118.982788  
0          239         125.668345  

[4578 rows x 7 columns]

In [27]:
viewer.fitness_evolution()

In [30]:
viewer.box_plot(3, 1000)

In [31]:
fig = viewer.parallel_coordinates(4000)
fig.update_layout(width=1000, height=700)